In [ ]:
import pandas as pd
from os.path import isfile, join
import numpy as np
import hydromt
import xarray as xr

In [ ]:
mdir = r"../../3_models"
rdir = r"../../4_results"

In [ ]:
# settings
hmin = 0.15


In [ ]:
# flood impact functions

def flood_damage(da_flddph, da_exposure, df_susceptibility, **kwargs):
    nodata = da_exposure.attrs['_FillValue']
    da0 = df_susceptibility.to_xarray()['factor']#.chunk({'depth':-1})
    factor = np.minimum(1, da0.interp(depth=np.minimum(da0.max(),da_flddph), **kwargs))
    damage = (factor * da_exposure).fillna(nodata).astype(np.float32)
    damage.name = da_exposure.name
    damage.attrs.update(**da_exposure.attrs)
    return damage

def flood_exposed(da_flddph, da_exposure, min_flddph=hmin):
    exposed = xr.where(da_flddph>min_flddph,da_exposure,0.0).astype(np.float32)
    exposed.attrs.update(**da_exposure.attrs)
    exposed.name = da_exposure.name
    return exposed

In [ ]:
# read basemodel
from hydromt_sfincs import SfincsModel
mod0 = SfincsModel(join(mdir, 'sfincs', '00_base_riv'), mode='r')
rivmsk = mod0.staticmaps['rivmsk']==1
mask0 = mod0.staticmaps['dep']==mod0.staticmaps['dep'].raster.nodata
mask = np.logical_or(rivmsk, mask0)

In [ ]:
# read vulnerability curves and 
df = pd.read_csv(join(mdir, 'fiat', 'susceptibility', 'AF000.csv'), index_col=0)
df.columns = ['factor']
df.index.name = 'depth'
# correct min flood depth (hmin)
df[df.index<=hmin] = 0

In [ ]:
# read exposure
ds_exp = hydromt.open_mfraster(join(mdir, 'fiat', 'exposure', '*.tif')).load()
ds_exp = ds_exp[['buildings_value', 'population_count']]

In [ ]:
# read scenario where we expect no flooding for bias correction
postfix = '_dt0'
da_bias = xr.open_rasterio(join(mdir, 'sfincs', f'qb000_qp000_h000_p000{postfix}', 'gis', 'hmax.tif')).squeeze(drop=True)
da_bias = np.maximum(0, da_bias).where(~mask)
da_bias.raster.set_nodata(np.nan)
# da_bias.raster.to_raster(join(rdir, 'hmax', f'bias{postfix}.tif'), compress='deflate')
# da_bias = da_bias.fillna(0)
# da_bias.where(~mask).plot(vmax=1.0)

In [ ]:
# prepare output
set_name = 'sim_SCEN_all_rps7'
df_scens = pd.read_csv(join(rdir, f'{set_name}.csv'), index_col=0)
if postfix:
    df_scens['scen'] = [f'{name}{postfix}' for name in df_scens['scen']]
df_scens['finished'] = False
df_scens.to_csv(join(rdir, f'impact_bias0{postfix}.csv'))

In [ ]:
# save univariate and full dep hazards

for i, row in df_scens.iterrows():
    scen = row['scen']
    hazard_fn = join(mdir, 'sfincs', scen, 'gis', 'hmax.tif')
    rps = row[['qb_rp', 'qp_rp', 'p_rp', 'h_tsw_rp']]
    # save rasters
    if np.all(np.diff(rps) == 0) or np.sum(rps > 0) == 1:
        da_hmax = xr.open_rasterio(hazard_fn).squeeze(drop=True)
        # correct for perm water and bias
        da_hmax = da_hmax.where(mask0, np.maximum(0, da_hmax - da_bias))
        # save
        da_hmax.raster.set_nodata(np.nan)
        da_hmax.raster.to_raster(join(rdir, 'hmax', f'{scen}.tif'), compress='deflate')

In [ ]:
ds_hmax = hydromt.open_mfraster(join(rdir, 'hmax', f'*{postfix}.tif')).load()
ds_hmax

In [ ]:
# zoning -> preprocess exposure based on fulldep rp2/5/10 zones
zoning = {}
for rp0 in [2,5,10]:
    flood_zone = xr.concat([
        ds_hmax[f'qb{rp0:03d}_qp000_h000_p000'],
        ds_hmax[f'qb000_qp{rp0:03d}_h000_p000'],
        ds_hmax[f'qb000_qp000_h{rp0:03d}_p000'],
        ds_hmax[f'qb000_qp000_h000_p{rp0:03d}'],
    ], dim='dvar').max('dvar') > hmin
    zoning[rp0] = ds_exp.copy().where(~flood_zone,0).compute()
    for dvar in ds_exp.data_vars.keys():
        print(rp0, dvar, (ds_exp[dvar].sum() - zoning[rp0][dvar].sum()).item())


In [ ]:
# dry proofing -> range of hmin [0.3-0.7] in vulnerability curves
dryproof = {}
for hmin0 in [0.5,0.75,0.999]:
    df0 = df.copy(deep=True)
    df0[df0.index<=hmin0] = 0
    dryproof[int(np.round(hmin0*100))] = df0

In [ ]:
# process per hazard simulatio
df_scens = pd.read_csv(join(rdir, f'impact_bias0{postfix}.csv'), index_col=0)#.head()

for i, row in df_scens.iterrows():
    scen = row['scen']
    hazard_fn = join(mdir, 'sfincs', scen, 'gis', 'hmax.tif')
    rps = row[['qb_rp', 'qp_rp', 'p_rp', 'h_tsw_rp']]
    qb_rp, qp_rp, p_rp, h_rp = rps
    if row['finished'] or not isfile(hazard_fn):
        continue 
    # read hazard
    da_hmax = xr.open_rasterio(hazard_fn).squeeze(drop=True)
    # correct for perm water and bias
    da_hmax = da_hmax.where(mask, da_hmax - da_bias)
    da_hmax = np.maximum(0, da_hmax).where(~mask)
    # impact assessment - base
    df_scens.loc[i, 'dam'] = flood_damage(da_hmax, ds_exp['buildings_value'], df).sum()
    df_scens.loc[i, 'ppl'] = flood_exposed(da_hmax, ds_exp['population_count'], hmin).sum()

    # impact assessment - adaptation scenarios
    # zoning
    for rp0 in zoning:
        df_scens.loc[i, f'dam_zoning{rp0:03d}'] = flood_damage(da_hmax, zoning[rp0]['buildings_value'], df).sum()
        df_scens.loc[i, f'ppl_zoning{rp0:03d}'] = flood_exposed(da_hmax, zoning[rp0]['population_count'], hmin).sum()
    # dry proofing
    for hmin0 in dryproof:
        df_scens.loc[i, f'dam_dryproof{hmin0}'] = flood_damage(da_hmax, ds_exp['buildings_value'], dryproof[hmin0]).sum()
        df_scens.loc[i, f'ppl_dryproof{hmin0}'] = flood_exposed(da_hmax, ds_exp['population_count'], hmin0/100).sum()
    # dikes -> remove q and h hazard; keep p hazard below or equal to rp5/10/50
    for rp0 in [5, 10, 50]:
        da_hmax1 = da_hmax.copy()
        if qb_rp <= rp0:
            da_hmax1 = da_hmax1.where(mask, np.maximum(0, da_hmax1 - ds_hmax[f'qb{qb_rp:03d}_qp000_h000_p000']))
        if qp_rp <= rp0:
            da_hmax1 = da_hmax1.where(mask, np.maximum(0, da_hmax1 - ds_hmax[f'qb000_qp{qp_rp:03d}_h000_p000']))
        if h_rp <= rp0:
            da_hmax1 = da_hmax1.where(mask, np.maximum(0, da_hmax1 - ds_hmax[f'qb000_qp000_h{h_rp:03d}_p000']))
        if p_rp > 0:
            da_hmax1 = da_hmax1.where(mask, np.maximum(da_hmax1, ds_hmax[f'qb000_qp000_h000_p{p_rp:03d}']))
        df_scens.loc[i, f'dam_dikes{rp0:03d}'] = flood_damage(da_hmax1, ds_exp['buildings_value'], df).sum()
        df_scens.loc[i, f'ppl_dikes{rp0:03d}'] = flood_exposed(da_hmax1, ds_exp['population_count'], hmin).sum()

    df_scens.loc[i, 'finished'] = True

# save to csv
df_scens = df_scens.round(1)
df_scens.to_csv(join(rdir, f'impact_bias0{postfix}.csv'))
df_scens[df_scens['finished']].head()